# Imports

In [40]:
import os
import numpy as np
import pandas as pd

# Constants

In [41]:
PROCESSED_FILE_DIRECTORY = r"..\data\excel"

REPORT_FILE_DIRECTORY = r"..\data\report"

# Get Files

In [42]:
files = os.listdir(PROCESSED_FILE_DIRECTORY)

files = [f for f in files if os.path.isfile(PROCESSED_FILE_DIRECTORY + '/' + f)]

# Data Processing

## Default Table Schema

In [43]:
def get_empty_dataframe(file, sheet_name):
    if sheet_name == "0":
        data_df = pd.DataFrame({'2021-22': [0],
                                            '2020-21': [0],
                                            '2019-20': [0],
                                            '2018-19': [0],
                                            '2017-18': [0],
                                            '2016-17': [0],
                                            'Institution': file.split('.')[0]
                                           })
    elif sheet_name == "1":
        data_df = pd.DataFrame({'Total Students': [0], 
                                            'Institution': file.split('.')[0]                    
                    })
    elif sheet_name == "2":
        data_df = pd.DataFrame({'Median salary UG': [0], 
                                            'Institution': file.split('.')[0]                    
                    })
    elif sheet_name == "3":
        data_df = pd.DataFrame({'Median salary PG': [0], 
                                            'Institution': file.split('.')[0]                    
                    })   
    elif sheet_name == "4":
        data_df = pd.DataFrame({'Total': [0], 
                                            'Institution': file.split('.')[0]
                    })    
    elif sheet_name == "5":
        data_df = pd.DataFrame({
                        '2021-22': [0],
                        '2020-21': [0],
                        '2019-20': [0],
                        'Institution': file.split('.')[0],
                        'Capex1': [0],
                        'Capex2': [0],
                        'Capex3': [0]
                    }) 
    elif sheet_name == "6":
        data_df = pd.DataFrame({
                        '2021-22': [0],
                        '2020-21': [0],
                        '2019-20': [0],
                        'Institution': file.split('.')[0],
                        'Opex1': [0],
                        'Opex2': [0],
                        'Opex3': [0]
                    })
    elif sheet_name == "7":
        data_df = pd.DataFrame({
                        'Sponsored Average': [0], 
                        'Institution': file.split('.')[0]
                    })  
    elif sheet_name == "8":
        data_df = pd.DataFrame({
                        'Consultancy Average': [0], 
                        'Institution': file.split('.')[0]
                    })    
    elif sheet_name == "10":
        data_df = pd.DataFrame({
                        'Answer': [None], 
                        'Institution': file.split('.')[0]
                    })        
    return data_df

## Sheet 0

In [44]:
def process_sheet0():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            if "1" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:            
                #print(f"Processing the sheet 0 of the file: {file}")

                excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                data_df = pd.read_excel(excel_file, sheet_name="0")
                data_df.replace('-', 0, inplace=True)
                data_df = data_df.iloc[:, 1:]

                data_df['2019-20'] = data_df['2019-20'].astype(np.int64)
                data_df['2018-19'] = data_df['2018-19'].astype(np.int64)
                data_df['2017-18'] = data_df['2017-18'].astype(np.int64)
                data_df['2016-17'] = data_df['2016-17'].astype(np.int64)

                data_df['Total'] = data_df['2021-22'] + data_df['2020-21'] + data_df['2019-20'] + \
                data_df['2018-19'] + data_df['2017-18'] + data_df['2016-17']    
            else:
                data_df = get_empty_dataframe(file, "0")
            
            data_df.fillna(0, inplace=True)
            nirf_df = pd.concat([nirf_df, data_df])           
    except Exception as ex:
        print(f"Error while processing the sheet 0 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file, "0")
    finally:
        nirf_df = nirf_df.groupby('Institution').sum('Total')[['Total']]
        nirf_df = nirf_df.transpose()
        nirf_df.rename(index={'Total': 'Total sanctioned approved intake for all programs'}, inplace=True)    

    return nirf_df

### Sheet 1

In [45]:
def process_sheet1():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            if "1" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                #print(f"Processing the sheet 1 of the file: {file}")

                excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                data_df = pd.read_excel(excel_file, sheet_name="1")

                data_df = data_df[['Total Students', 'Institution']]
            else:
                data_df = get_empty_dataframe(file, "1")
                
            data_df.fillna(0, inplace=True)        
            nirf_df = pd.concat([nirf_df, data_df])                        
    except Exception as ex:
        print(f"Error while processing the sheet 1 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file, "1") 
    finally:                
        nirf_df = nirf_df.groupby('Institution').sum('Total Students')
        nirf_df = nirf_df.transpose()
        nirf_df.rename(index={'Total Students': 'Total number of students enrolled in all programs'}, inplace=True)        
    
    return nirf_df

## Sheet 2

In [46]:
def process_sheet2():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            try:
                if "2" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                    #print(f"Processing the sheet 2 of the file: {file}")

                    excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                    data_df = pd.read_excel(excel_file, sheet_name="2")
                    
                    columns = data_df.columns.to_list()
                    median_index = 0
                    institute_index = 0
                    
                    for index, element in enumerate(columns):
                        if element.split()[0] == 'Median':
                            median_index = index                            
                        elif element.split()[0] == 'Institution':
                            institute_index = index

                    data_df = pd.DataFrame({
                        'Median salary UG': data_df.iloc[:, median_index].values,
                        'Institution': data_df.iloc[:, institute_index].values
                    })

                    data_df['Median salary UG'] = data_df['Median salary UG'].str.split(r"(").str[0].astype(np.float64)
                else:
                    data_df = get_empty_dataframe(file, "2")
            except Exception as ex:
                data_df = get_empty_dataframe(file, "2")
            finally:  
                data_df.fillna(0, inplace=True)
                nirf_df = pd.concat([nirf_df, data_df])           
    except Exception as ex:
        print(f"Error while processing the sheet 2 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file, "2")
    finally:            
        nirf_df = nirf_df.groupby('Institution').mean('Median salary UG')
        nirf_df = nirf_df.transpose()           
    
    return nirf_df

## Sheet 3

In [47]:
def process_sheet3():
    nirf_df = pd.DataFrame()

    #try:
    for file in files:
        try:
            if "3" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                #print(f"Processing the sheet 2 of the file: {file}")

                excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                data_df = pd.read_excel(excel_file, sheet_name="3")
                    
                columns = data_df.columns.to_list()
                median_index = 0
                institute_index = 0
                    
                for index, element in enumerate(columns):
                    if element.split()[0] == 'Median':
                        median_index = index
                    elif element.split()[0] == 'Institution':
                        institute_index = index

                data_df = pd.DataFrame({
                        'Median salary PG': data_df.iloc[:, median_index].values,
                        'Institution': data_df.iloc[:, institute_index].values
                    })

                data_df['Median salary PG'] = data_df['Median salary PG'].str.split(r"(").str[0].astype(np.float64)
            else:
                data_df = get_empty_dataframe(file, "3")
        except Exception as ex:
            #print(f"Error while processing the sheet 3 of the file: {file}")
            #print(str(ex))
            data_df = get_empty_dataframe(file, "3")
        finally:    
            data_df.fillna(0, inplace=True)
            nirf_df = pd.concat([nirf_df, data_df])
            continue
    '''
    except Exception as ex:
        print(f"Error while processing the sheet 3 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file, "3") 
    finally: 
    '''
    nirf_df.fillna(0, inplace=True)
    nirf_df = nirf_df.groupby('Institution').mean('Median salary PG')
    nirf_df = nirf_df.transpose() 
    
    return nirf_df

In [48]:
process_sheet3()

Institution,IR-O-C-16604,IR-O-I-1074,IR-O-I-1075,IR-O-I-1357,IR-O-I-1441,IR-O-N-10,IR-O-N-15,IR-O-U-0020,IR-O-U-0042,IR-O-U-0053,...,IR-O-U-0560,IR-O-U-0564,IR-O-U-0570,IR-O-U-0572,IR-O-U-0575,IR-O-U-0577,IR-O-U-0584,IR-O-U-0642,IR-O-U-0701,IR-O-U-0747
Median salary PG,503333.333333,858571.428571,1.554333e+06,240703.666667,0.0,0.0,0.0,435166.666667,181818.181818,1300000.0,...,272727.272727,0.0,0.0,240000.0,0.0,62727.272727,140000.0,0.0,692285.714286,0.0


## Sheet 4

In [49]:
def process_sheet4():
    nirf_df = pd.DataFrame()
    
    try:    
        for file in files:
            if "4" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                #print(f"Processing the sheet 4 of the file: {file}")

                excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                data_df = pd.read_excel(excel_file, sheet_name="4")

                data_df.rename(columns={'Unnamed: 0': 'Total'}, inplace=True)
                data_df = data_df.iloc[1:3, [1, 4]]
                data_df['Total'] = data_df['Total'].astype(np.int64) 
            else:
                data_df = get_empty_dataframe(file, "4")
                
            data_df.fillna(0, inplace=True)        
            nirf_df = pd.concat([nirf_df, data_df])            
    except Exception as ex:
        print(f"Error while processing the sheet 4 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file, "4")
    finally:            
        nirf_df = nirf_df.groupby('Institution').sum('Total')
        nirf_df = nirf_df.transpose()
        nirf_df.rename(index={'Total': 'Total number of PhD students enrolled'}, inplace=True)
    
    return nirf_df

In [50]:
def process_sheet4_1():
    nirf_df = pd.DataFrame()
    
    try:    
        for file in files:
            if "4" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                #print(f"Processing the sheet 4 of the file: {file}")

                excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                data_df = pd.read_excel(excel_file, sheet_name="4")

                data_df = data_df.iloc[[5], 1:]
                
                data_df['Unnamed: 0'] = data_df['Unnamed: 0'].astype(np.int64)
                data_df['Unnamed: 1'] = data_df['Unnamed: 1'].astype(np.int64)
                data_df['Unnamed: 2'] = data_df['Unnamed: 2'].astype(np.int64)
                
                data_df['Total'] = (data_df['Unnamed: 0'] + data_df['Unnamed: 1'] + data_df['Unnamed: 2'])/3              
                data_df = data_df[['Total', 'Institution']]
            else:
                data_df = get_empty_dataframe(file, "4")
                    
            data_df.fillna(0, inplace=True)
            nirf_df = pd.concat([nirf_df, data_df])            
    except Exception as ex:
        print(f"Error while processing the sheet 4 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file, "4") 
    finally:            
        nirf_df = nirf_df.set_index('Institution')
        nirf_df.sort_index(inplace=True)
        nirf_df = nirf_df.transpose()
        nirf_df.rename(index={'Total': 'Average Full Time PhD students graduated'}, inplace=True)
    
    return nirf_df

In [51]:
def process_sheet4_2():
    nirf_df = pd.DataFrame()
    
    try:    
        for file in files:
            if "4" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                #print(f"Processing the sheet 4 of the file: {file}")

                excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                data_df = pd.read_excel(excel_file, sheet_name="4")

                data_df = data_df.iloc[[6], 1:]
                
                data_df['Unnamed: 0'] = data_df['Unnamed: 0'].astype(np.int64)
                data_df['Unnamed: 1'] = data_df['Unnamed: 1'].astype(np.int64)
                data_df['Unnamed: 2'] = data_df['Unnamed: 2'].astype(np.int64)
                
                data_df['Total'] = (data_df['Unnamed: 0'] + data_df['Unnamed: 1'] + data_df['Unnamed: 2'])/3              
                data_df = data_df[['Total', 'Institution']]
                data_df.fillna(0, inplace=True)
            else:
                data_df = get_empty_dataframe(file, "4")
                
            data_df.fillna(0, inplace=True)        
            nirf_df = pd.concat([nirf_df, data_df])            
    except Exception as ex:
        print(f"Error while processing the sheet 4 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file, "4")    
    finally:            
        nirf_df = nirf_df.set_index('Institution')
        nirf_df.sort_index(inplace=True)
        nirf_df = nirf_df.transpose()
        nirf_df.rename(index={'Total': 'Average Part Time PhD students graduated'}, inplace=True)
    
    return nirf_df

## Sheet 5

In [52]:
def process_sheet5():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            if "5" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                #print(f"Processing the sheet 5 of the file: {file}")

                excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                data_df = pd.read_excel(excel_file, sheet_name="5")

                data_df = data_df.iloc[2:, 1:]

                data_df['2021-22'] = data_df['2021-22'].str.split(r"(").str[0].astype(np.int64)

                data_df['2020-21'] = data_df['2020-21'].str.split(r"(").str[0].astype(np.int64)

                data_df['2019-20'] = data_df['2019-20'].str.split(r"(").str[0].astype(np.int64)

                data_df['Capex1'] = round((np.sum(data_df['2021-22'])/100000),1)

                data_df['Capex2'] = round((np.sum(data_df['2020-21'])/100000),1)

                data_df['Capex3'] = round((np.sum(data_df['2019-20'])/100000),1)
            else:
                data_df = get_empty_dataframe(file, "5")
                
            data_df.fillna(0, inplace=True)        
            nirf_df = pd.concat([nirf_df, data_df])       
    except Exception as ex:
        print(f"Error while processing the sheet 5 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file, "5")     
    finally:            
        nirf_df = nirf_df[['Capex1', 'Capex2', 'Capex3', 'Institution']].sort_values('Institution')
        nirf_df = nirf_df.set_index('Institution')
        nirf_df = nirf_df.groupby('Institution').head(1)
        nirf_df = nirf_df.transpose()      
    
    return nirf_df

## Sheet 6

In [53]:
def process_sheet6():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            if "6" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                #print(f"Processing the sheet 6 of the file: {file}")
                    
                excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                data_df = pd.read_excel(excel_file, sheet_name="6")

                data_df = data_df.iloc[2:, 1:]

                data_df['2021-22'] = data_df['2021-22'].str.split(r"(").str[0].astype(np.int64)

                data_df['2020-21'] = data_df['2020-21'].str.split(r"(").str[0].astype(np.int64)

                data_df['2019-20'] = data_df['2019-20'].str.split(r"(").str[0].astype(np.int64)

                data_df['Opex1'] = round((np.sum(data_df['2021-22'])/1000000),1)

                data_df['Opex2'] = round((np.sum(data_df['2020-21'])/1000000),1)

                data_df['Opex3'] = round((np.sum(data_df['2019-20'])/1000000),1)                
            else:
                data_df = get_empty_dataframe(file, "6")
                
            data_df.fillna(0, inplace=True)     
            nirf_df = pd.concat([nirf_df, data_df])           
    except Exception as ex:
        print(f"Error while processing the sheet 6 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file, "6")
    finally:            
        nirf_df = nirf_df[['Opex1', 'Opex2', 'Opex3', 'Institution']].sort_values('Institution')
        nirf_df = nirf_df.set_index('Institution')
        nirf_df = nirf_df.groupby('Institution').head(1)
        nirf_df = nirf_df.transpose() 
    
    return nirf_df

## Sheet 7

In [54]:
def process_sheet7():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            if "7" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                #print(f"Processing the sheet 7 of the file: {file}")

                excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                data_df = pd.read_excel(excel_file, sheet_name="7")

                data_df = data_df.iloc[[2], 1:]
                data_df['2021-22'] = data_df['2021-22'].astype(np.float64)
                data_df['2020-21'] = data_df['2020-21'].astype(np.float64)
                data_df['2019-20'] = data_df['2019-20'].astype(np.float64)

                data_df['Sponsored Average'] = (data_df['2021-22'] + data_df['2020-21'] + data_df['2019-20'])/3
                data_df = data_df[['Sponsored Average', 'Institution']]
            else:
                data_df = get_empty_dataframe(file, "7")
                
            data_df.fillna(0, inplace=True)
            nirf_df = pd.concat([nirf_df, data_df])
    except Exception as ex:
        print(f"Error while processing the sheet 7 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file, "7")
    finally:            
        nirf_df = nirf_df.groupby('Institution').mean('Sponsored Average')
        nirf_df = nirf_df.transpose() 
    
    return nirf_df

## Sheet 8

In [55]:
def process_sheet8():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            if "8" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                #print(f"Processing the sheet 8 of the file: {file}")

                excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                data_df = pd.read_excel(excel_file, sheet_name="8")

                data_df = data_df.iloc[[2], 1:]
                data_df['2021-22'] = data_df['2021-22'].astype(np.float64)
                data_df['2020-21'] = data_df['2020-21'].astype(np.float64)
                data_df['2019-20'] = data_df['2019-20'].astype(np.float64)

                data_df['Consultancy Average'] = (data_df['2021-22'] + data_df['2020-21'] + data_df['2019-20'])/3
                data_df = data_df[['Consultancy Average', 'Institution']]
            else:
                data_df = get_empty_dataframe(file, "8")
                
            data_df.fillna(0, inplace=True)        
            nirf_df = pd.concat([nirf_df, data_df])
    except Exception as ex:
        print(f"Error while processing the sheet 8 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file, "8") 
    finally:        
        nirf_df = nirf_df.groupby('Institution').mean('Consultancy Average')
        nirf_df = nirf_df.transpose()
    
    return nirf_df

## Sheet 10

In [56]:
def process_sheet10():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            if "10" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                #print(f"Processing the sheet {sheet_name} of the file: {file}")
                    
                excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                data_df = pd.read_excel(excel_file, sheet_name="10")
            else:
                data_df = get_empty_dataframe(file, "10")
                
            data_df.fillna(0, inplace=True)    
            nirf_df = pd.concat([nirf_df, data_df])
    except Exception as ex:
        print(f"Error while processing the sheet 10 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file, "10")
    finally:
        nirf_df = nirf_df[['Answer', 'Institution']].sort_values('Institution')
        nirf_df = nirf_df.set_index('Institution')
        nirf_df.sort_index(inplace=True)
        nirf_df = nirf_df.transpose()
        nirf_df.rename(index={'Answer': 'Number of faculty members'}, inplace=True)
        
    return nirf_df

In [57]:
process_sheet10()

Institution,IR-O-C-16604,IR-O-I-1074,IR-O-I-1075,IR-O-I-1357,IR-O-I-1441,IR-O-N-10,IR-O-N-15,IR-O-U-0020,IR-O-U-0042,IR-O-U-0053,...,IR-O-U-0560,IR-O-U-0564,IR-O-U-0570,IR-O-U-0572,IR-O-U-0575,IR-O-U-0577,IR-O-U-0584,IR-O-U-0642,IR-O-U-0701,IR-O-U-0747
Number of faculty members,273,0,597,372,788,554,734,971,421,536,...,622,997,1255,136,902,283,284,248,365,2658


In [58]:
nirf_df = pd.DataFrame()

data = process_sheet0()
nirf_df = pd.concat([nirf_df, data])

data = process_sheet1()
nirf_df = pd.concat([nirf_df, data])

data = process_sheet2()
nirf_df = pd.concat([nirf_df, data])

data = process_sheet3()
nirf_df = pd.concat([nirf_df, data])

nirf_df = nirf_df.transpose()
nirf_df['Median Salary'] = (nirf_df['Median salary UG'] + nirf_df['Median salary PG'])/2
nirf_df = nirf_df.transpose()

data = process_sheet4()
nirf_df = pd.concat([nirf_df, data])

data = process_sheet4_1()
nirf_df = pd.concat([nirf_df, data])

data = process_sheet4_2()
nirf_df = pd.concat([nirf_df, data])

data = process_sheet5()
nirf_df = pd.concat([nirf_df, data])

nirf_df = nirf_df.transpose()
nirf_df['capexperstudavg'] = (nirf_df['Capex1'] + \
                              nirf_df['Capex2'] + nirf_df['Capex3'])/ \
(3 * (nirf_df['Total sanctioned approved intake for all programs'] + nirf_df['Total number of PhD students enrolled']))
nirf_df = nirf_df.transpose()

data = process_sheet6()
nirf_df = pd.concat([nirf_df, data])

nirf_df = nirf_df.transpose()
nirf_df['opexperstudavg'] = (nirf_df['Opex1'] + \
                              nirf_df['Opex2'] + nirf_df['Opex3'])/ \
(3 * (nirf_df['Total sanctioned approved intake for all programs'] + nirf_df['Total number of PhD students enrolled']))
nirf_df = nirf_df.transpose()

data = process_sheet7()
nirf_df = pd.concat([nirf_df, data])

data = process_sheet8()
nirf_df = pd.concat([nirf_df, data])

data = process_sheet10()
nirf_df = pd.concat([nirf_df, data])


nirf_df

Error while processing the sheet 0 of the file: IR-O-I-1441.xlsx
unsupported operand type(s) for +: 'int' and 'str'


KeyError: "None of [Index(['Total'], dtype='object')] are in the [columns]"

In [ ]:
#nirf_df.to_excel(REPORT_FILE_DIRECTORY + "/PredictionReport.xlsx", freeze_panes=(1, 1),)